In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
import h5py

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from uncluster.config import t_max
from uncluster.potential import mw_potential
from uncluster.cluster_massloss import solve_mass_evolution
from uncluster.utils import quantity_from_hdf5

In [ ]:
# Load the initial conditions
with h5py.File('../cache/sph_iso.hdf5', 'a') as root:
    f = root['progenitors/initial']

    gc_masses = quantity_from_hdf5(f['mass'])
    gc_w0 = gd.CartesianPhaseSpacePosition(pos=quantity_from_hdf5(f['pos']),
                                           vel=quantity_from_hdf5(f['vel']))
    n_clusters = len(gc_masses)

In [ ]:
H = gp.Hamiltonian(mw_potential)

In [ ]:
idx = 8

w0 = gc_w0[idx]
initial_mass = gc_masses[idx]
print("r={:.2f}, m0={:.2e}".format(w0.spherical[0].r[0], initial_mass))

gc_orbit = H.integrate_orbit(w0, n_steps=16384, # MAGIC NUMBER
                             t1=t_max, t2=0.,
                             Integrator=gi.DOPRI853Integrator)

In [ ]:
_ = gc_orbit.plot(marker='')

In [ ]:
# solve for the mass-loss history of the cluster over the integration grid
t_grid = gc_orbit.t
r_grid = gc_orbit.r

# solve dM/dt to get mass-loss history
disrupt_idx, mass_grid = solve_mass_evolution(initial_mass.to(u.Msun).value,
                                              t_grid.to(u.Gyr).value,
                                              r_grid.to(u.kpc).value)

In [ ]:
plt.plot(t_grid, mass_grid, marker='')